In [36]:
import pandas as pd

In [39]:
df = pd.read_csv("/data/smsSpamCollection.arff", quotechar="'", escapechar="\\", header = None, names=["body", "spam"])

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5574 entries, 0 to 5573
Data columns (total 2 columns):
body    5574 non-null object
spam    5574 non-null int64
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [41]:
df.to_json("/data/sms.json")

In [42]:
pd.read_json("/data/sms.json")

,body,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
10,I'm gonna be home soon and i don't want to tal...,0
100,Please don't text me anymore. I have nothing e...,0
1000,"Aight will do, thanks again for comin out",0
1001,No..but heard abt tat..,0
1002,Please call our customer service representativ...,1
1003,Yes..he is really great..bhaji told kallis bes...,0
1004,&lt;#&gt; am I think? Should say on syllabus,0
1005,Umma. Did she say anything,0


In [44]:
df.spam.value_counts()/len(df)

0    0.865985
1    0.134015
Name: spam, dtype: float64